In [1]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.ml.linalg import Vectors

In [2]:
# 開啟最佳化 Spark 效能
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

In [71]:
sdf_3s = spark.read.csv('allmusic3s.csv',sep=",",header=True, inferSchema=True)
sdf_3s.createOrReplaceTempView("dfTable")

In [69]:
sdf_3s = sdf_3s.dropna(how='any')

In [ ]:
sdf_3s.printSchema()

In [ ]:
sdf_3s.count()

In [ ]:
type(sdf_3s)

In [ ]:
sdf_3s.select("total_beats").show()

***

In [ ]:
# 測試可執行
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

cols = sdf_3s.columns
feature_sdf_3s = sdf_3s.select(cols[5:8]) # ['chroma_stft_mean', 'chroma_stft_std', 'chroma_stft_var']
cols = feature_sdf_3s.columns

assembler = VectorAssembler(inputCols=cols, outputCol="features")
output = assembler.transform(feature_sdf_3s)
# output.show()
scaler = MinMaxScaler(inputCol='features', outputCol="scaledFeatures")
# rescale each feature to range [min, max].
scaledData = scaler.fit(output).transform(output)
scaledData.show()

***

In [10]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, StandardScaler, MinMaxScaler,VectorAssembler
from pyspark.ml import Pipeline

In [70]:
# 刪除多餘欄位
sdf_cols = sdf_3s.columns
select_sdf_3s = sdf_3s.select(sdf_cols[4:])
select_sdf_3s.printSchema()

root
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = true)
 |-- _c28: string (nullable = true)
 |-- _c29: string (nullable = true)
 |-- _c30: string (nullable = true)
 |-- _c31: string (nullable =

In [15]:
continuous_features = [d[0] for d in select_sdf_3s.dtypes if (d[1] != 'string')]
# continuous_features = select_sdf_3s.select("")

In [17]:
# label 編碼
# 選取需要的標籤 input="label", output="y"
indexers = [StringIndexer(inputCol="label", outputCol="y")] 

# 特徵轉向量
# input = for迴圈取出所有column 並[]轉成 list, output= "features" 一個標籤
assemblers = VectorAssembler(inputCols=[col for col in continuous_features], outputCol="features")

# MinMixScaler 標準化
# input= "features"(Vector column), output="mmfeartures"
mmScalers = MinMaxScaler(inputCol="features", outputCol="mmfeatures")

# Piplin整合一起執行
pipeline = Pipeline(stages= [assemblers, mmScalers] + indexers)
scalerModel = pipeline.fit(select_sdf_3s) 
scaledData = scalerModel.transform(select_sdf_3s)

22/10/04 15:28:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , song_name, tempo, total_beats, average_beats, chroma_stft_mean, chroma_stft_std, chroma_stft_var, chroma_cq_mean, chroma_cq_std, chroma_cq_var, chroma_cens_mean, chroma_cens_std, chroma_cens_var, melspectrogram_mean, melspectrogram_std, melspectrogram_var, mfcc1_mean, mfcc2_mean, mfcc3_mean, mfcc4_mean, mfcc5_mean, mfcc6_mean, mfcc7_mean, mfcc8_mean, mfcc9_mean, mfcc10_mean, mfcc11_mean, mfcc12_mean, mfcc13_mean, mfcc14_mean, mfcc15_mean, mfcc16_mean, mfcc17_mean, mfcc18_mean, mfcc19_mean, mfcc20_mean, mfcc1_std, mfcc2_std, mfcc3_std, mfcc4_std, mfcc5_std, mfcc6_std, mfcc7_std, mfcc8_std, mfcc9_std, mfcc10_std, mfcc11_std, mfcc12_std, mfcc13_std, mfcc14_std, mfcc15_std, mfcc16_std, mfcc17_std, mfcc18_std, mfcc19_std, mfcc20_std, mfcc1_var, mfcc2_var, mfcc3_var, mfcc4_var, mfcc5_var, mfcc6_var, mfcc7_var, mfcc8_var, mfcc9_var, mfcc10_var, mfcc11_var, mfcc12_var, mfcc13_var, mfcc14_var, mfcc15_

22/10/04 15:28:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , song_name, tempo, total_beats, average_beats, chroma_stft_mean, chroma_stft_std, chroma_stft_var, chroma_cq_mean, chroma_cq_std, chroma_cq_var, chroma_cens_mean, chroma_cens_std, chroma_cens_var, melspectrogram_mean, melspectrogram_std, melspectrogram_var, mfcc1_mean, mfcc2_mean, mfcc3_mean, mfcc4_mean, mfcc5_mean, mfcc6_mean, mfcc7_mean, mfcc8_mean, mfcc9_mean, mfcc10_mean, mfcc11_mean, mfcc12_mean, mfcc13_mean, mfcc14_mean, mfcc15_mean, mfcc16_mean, mfcc17_mean, mfcc18_mean, mfcc19_mean, mfcc20_mean, mfcc1_std, mfcc2_std, mfcc3_std, mfcc4_std, mfcc5_std, mfcc6_std, mfcc7_std, mfcc8_std, mfcc9_std, mfcc10_std, mfcc11_std, mfcc12_std, mfcc13_std, mfcc14_std, mfcc15_std, mfcc16_std, mfcc17_std, mfcc18_std, mfcc19_std, mfcc20_std, mfcc1_var, mfcc2_var, mfcc3_var, mfcc4_var, mfcc5_var, mfcc6_var, mfcc7_var, mfcc8_var, mfcc9_var, mfcc10_var, mfcc11_var, mfcc12_var, mfcc13_var, mfcc14_var, mfcc15_

In [19]:
scaledData.printSchema()

root
 |-- average_beats: double (nullable = true)
 |-- chroma_stft_mean: double (nullable = true)
 |-- chroma_stft_std: double (nullable = true)
 |-- chroma_stft_var: double (nullable = true)
 |-- chroma_cq_mean: double (nullable = true)
 |-- chroma_cq_std: double (nullable = true)
 |-- chroma_cq_var: double (nullable = true)
 |-- chroma_cens_mean: double (nullable = true)
 |-- chroma_cens_std: double (nullable = true)
 |-- chroma_cens_var: double (nullable = true)
 |-- melspectrogram_mean: double (nullable = true)
 |-- melspectrogram_std: double (nullable = true)
 |-- melspectrogram_var: double (nullable = true)
 |-- mfcc1_mean: double (nullable = true)
 |-- mfcc2_mean: double (nullable = true)
 |-- mfcc3_mean: double (nullable = true)
 |-- mfcc4_mean: double (nullable = true)
 |-- mfcc5_mean: double (nullable = true)
 |-- mfcc6_mean: double (nullable = true)
 |-- mfcc7_mean: double (nullable = true)
 |-- mfcc8_mean: double (nullable = true)
 |-- mfcc9_mean: double (nullable = true)
 

In [25]:
scaledData.groupby("label","y").count().show()

22/10/04 15:34:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , song_name, tempo, total_beats, average_beats, chroma_stft_mean, chroma_stft_std, chroma_stft_var, chroma_cq_mean, chroma_cq_std, chroma_cq_var, chroma_cens_mean, chroma_cens_std, chroma_cens_var, melspectrogram_mean, melspectrogram_std, melspectrogram_var, mfcc1_mean, mfcc2_mean, mfcc3_mean, mfcc4_mean, mfcc5_mean, mfcc6_mean, mfcc7_mean, mfcc8_mean, mfcc9_mean, mfcc10_mean, mfcc11_mean, mfcc12_mean, mfcc13_mean, mfcc14_mean, mfcc15_mean, mfcc16_mean, mfcc17_mean, mfcc18_mean, mfcc19_mean, mfcc20_mean, mfcc1_std, mfcc2_std, mfcc3_std, mfcc4_std, mfcc5_std, mfcc6_std, mfcc7_std, mfcc8_std, mfcc9_std, mfcc10_std, mfcc11_std, mfcc12_std, mfcc13_std, mfcc14_std, mfcc15_std, mfcc16_std, mfcc17_std, mfcc18_std, mfcc19_std, mfcc20_std, mfcc1_var, mfcc2_var, mfcc3_var, mfcc4_var, mfcc5_var, mfcc6_var, mfcc7_var, mfcc8_var, mfcc9_var, mfcc10_var, mfcc11_var, mfcc12_var, mfcc13_var, mfcc14_var, mfcc15_

+---------+---+-----+
|    label|  y|count|
+---------+---+-----+
|classical|3.0| 4094|
|    blues|2.0| 4480|
|     jazz|4.0| 4060|
|  country|6.0| 2765|
|   hiphop|7.0| 2399|
|   reggae|5.0| 3238|
|      pop|8.0| 2339|
|     rock|1.0| 4559|
|    metal|0.0| 5107|
+---------+---+-----+



***

In [ ]:
# 以 sklearn思維轉換一張相同形狀的表格，在 pyspark作法要一個一個欄位做轉換，是不正確的，
# 而且在 spark ML並不是這樣做。
sdf_cols = sdf_3s.columns
select_sdf_3s = sdf_3s.select(cols[4:-2])
columns_to_scale = select_sdf_3s.columns
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale]
scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_scale]
pipeline = Pipeline(stages=assemblers + scalers)
scalerModel = pipeline.fit(select_sdf_3s)
scaledData = scalerModel.transform(select_sdf_3s)

In [ ]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

In [ ]:
# 將所有特徵轉為一個向量的欄位叫 "features"再進 MinMaxScaler(inputCol="features")
# 輸出另一個 outputCol 欄位。
cols = sdf_3s.columns
select_sdf_3s2 = sdf_3s.select(cols[4:-2])
columns_to_scale = select_sdf_3s2.columns
assemblers2 = VectorAssembler(inputCols=[col for col in columns_to_scale], outputCol="features") 
scalers2 = MinMaxScaler(inputCol="features", outputCol="mmfeatures")
pipeline2 = Pipeline(stages=[assemblers2, scalers2])
scalerModel2 = pipeline2.fit(select_sdf_3s2)
scaledData2 = scalerModel2.transform(select_sdf_3s2)

In [42]:
scaledData.select("label").show(10,False)

22/10/04 16:34:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , song_name, tempo, total_beats, average_beats, chroma_stft_mean, chroma_stft_std, chroma_stft_var, chroma_cq_mean, chroma_cq_std, chroma_cq_var, chroma_cens_mean, chroma_cens_std, chroma_cens_var, melspectrogram_mean, melspectrogram_std, melspectrogram_var, mfcc1_mean, mfcc2_mean, mfcc3_mean, mfcc4_mean, mfcc5_mean, mfcc6_mean, mfcc7_mean, mfcc8_mean, mfcc9_mean, mfcc10_mean, mfcc11_mean, mfcc12_mean, mfcc13_mean, mfcc14_mean, mfcc15_mean, mfcc16_mean, mfcc17_mean, mfcc18_mean, mfcc19_mean, mfcc20_mean, mfcc1_std, mfcc2_std, mfcc3_std, mfcc4_std, mfcc5_std, mfcc6_std, mfcc7_std, mfcc8_std, mfcc9_std, mfcc10_std, mfcc11_std, mfcc12_std, mfcc13_std, mfcc14_std, mfcc15_std, mfcc16_std, mfcc17_std, mfcc18_std, mfcc19_std, mfcc20_std, mfcc1_var, mfcc2_var, mfcc3_var, mfcc4_var, mfcc5_var, mfcc6_var, mfcc7_var, mfcc8_var, mfcc9_var, mfcc10_var, mfcc11_var, mfcc12_var, mfcc13_var, mfcc14_var, mfcc15_

***

In [56]:
#切分訓練、測試資料
data = scaledData.select("features","mmfeatures", "y")
trainingData, testData = data.randomSplit([0.8, 0.2])

## ML Part

In [74]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier

In [ ]:
from pyspark.ml.feature.BucketedRandomProjectionLSHModel.approxNearestNeighbors

In [75]:
def select_model(algo, train, test):
    model = algo.fit(train)
    predictions = model.transform(test)
    test_result = model.evaluate(test)
    print('{} Accuracy: {:5.3f}'.format(type(algo).__name__, test_result.accuracy))
    # pred = predictions.select('label', 'prediction')
    # probabilities = predictions.select('label', 'probability')

In [77]:
#LogisticRegression
logr = LogisticRegression(featuresCol='mmfeatures',labelCol='y')
select_model(logr, trainingData, testData)

#RandomForest
rf = RandomForestClassifier(featuresCol='mmfeatures',labelCol='y')
# select_model(rf, trainingData, testData)

#GBTClassifier
gbt = GBTClassifier(featuresCol='mmfeatures',labelCol='y')
# select_model(gbt, trainingData, testData)


22/10/04 17:58:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , song_name, tempo, total_beats, average_beats, chroma_stft_mean, chroma_stft_std, chroma_stft_var, chroma_cq_mean, chroma_cq_std, chroma_cq_var, chroma_cens_mean, chroma_cens_std, chroma_cens_var, melspectrogram_mean, melspectrogram_std, melspectrogram_var, mfcc1_mean, mfcc2_mean, mfcc3_mean, mfcc4_mean, mfcc5_mean, mfcc6_mean, mfcc7_mean, mfcc8_mean, mfcc9_mean, mfcc10_mean, mfcc11_mean, mfcc12_mean, mfcc13_mean, mfcc14_mean, mfcc15_mean, mfcc16_mean, mfcc17_mean, mfcc18_mean, mfcc19_mean, mfcc20_mean, mfcc1_std, mfcc2_std, mfcc3_std, mfcc4_std, mfcc5_std, mfcc6_std, mfcc7_std, mfcc8_std, mfcc9_std, mfcc10_std, mfcc11_std, mfcc12_std, mfcc13_std, mfcc14_std, mfcc15_std, mfcc16_std, mfcc17_std, mfcc18_std, mfcc19_std, mfcc20_std, mfcc1_var, mfcc2_var, mfcc3_var, mfcc4_var, mfcc5_var, mfcc6_var, mfcc7_var, mfcc8_var, mfcc9_var, mfcc10_var, mfcc11_var, mfcc12_var, mfcc13_var, mfcc14_var, mfcc15_

22/10/04 17:58:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , song_name, tempo, total_beats, average_beats, chroma_stft_mean, chroma_stft_std, chroma_stft_var, chroma_cq_mean, chroma_cq_std, chroma_cq_var, chroma_cens_mean, chroma_cens_std, chroma_cens_var, melspectrogram_mean, melspectrogram_std, melspectrogram_var, mfcc1_mean, mfcc2_mean, mfcc3_mean, mfcc4_mean, mfcc5_mean, mfcc6_mean, mfcc7_mean, mfcc8_mean, mfcc9_mean, mfcc10_mean, mfcc11_mean, mfcc12_mean, mfcc13_mean, mfcc14_mean, mfcc15_mean, mfcc16_mean, mfcc17_mean, mfcc18_mean, mfcc19_mean, mfcc20_mean, mfcc1_std, mfcc2_std, mfcc3_std, mfcc4_std, mfcc5_std, mfcc6_std, mfcc7_std, mfcc8_std, mfcc9_std, mfcc10_std, mfcc11_std, mfcc12_std, mfcc13_std, mfcc14_std, mfcc15_std, mfcc16_std, mfcc17_std, mfcc18_std, mfcc19_std, mfcc20_std, mfcc1_var, mfcc2_var, mfcc3_var, mfcc4_var, mfcc5_var, mfcc6_var, mfcc7_var, mfcc8_var, mfcc9_var, mfcc10_var, mfcc11_var, mfcc12_var, mfcc13_var, mfcc14_var, mfcc15_

22/10/04 17:59:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , song_name, tempo, total_beats, average_beats, chroma_stft_mean, chroma_stft_std, chroma_stft_var, chroma_cq_mean, chroma_cq_std, chroma_cq_var, chroma_cens_mean, chroma_cens_std, chroma_cens_var, melspectrogram_mean, melspectrogram_std, melspectrogram_var, mfcc1_mean, mfcc2_mean, mfcc3_mean, mfcc4_mean, mfcc5_mean, mfcc6_mean, mfcc7_mean, mfcc8_mean, mfcc9_mean, mfcc10_mean, mfcc11_mean, mfcc12_mean, mfcc13_mean, mfcc14_mean, mfcc15_mean, mfcc16_mean, mfcc17_mean, mfcc18_mean, mfcc19_mean, mfcc20_mean, mfcc1_std, mfcc2_std, mfcc3_std, mfcc4_std, mfcc5_std, mfcc6_std, mfcc7_std, mfcc8_std, mfcc9_std, mfcc10_std, mfcc11_std, mfcc12_std, mfcc13_std, mfcc14_std, mfcc15_std, mfcc16_std, mfcc17_std, mfcc18_std, mfcc19_std, mfcc20_std, mfcc1_var, mfcc2_var, mfcc3_var, mfcc4_var, mfcc5_var, mfcc6_var, mfcc7_var, mfcc8_var, mfcc9_var, mfcc10_var, mfcc11_var, mfcc12_var, mfcc13_var, mfcc14_var, mfcc15_

LogisticRegression Accuracy: 0.754
